In [1]:
from google.colab import drive
drive.mount('/content/drive')
!cd /content/drive/My\ Drive/gc_projects/Handwriten_annotation
project_dir = '/content/drive/My Drive/gc_projects/Handwriten_annotation/'
import os
os.chdir(project_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# profile
! npm install -g localtunnel
LOG_DIR = 'profile/'

import shutil
import os

if os.path.exists(LOG_DIR):
  shutil.rmtree(LOG_DIR)
os.mkdir(LOG_DIR)

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

! pip install --upgrade "cloud-tpu-profiler>=1.12"

get_ipython().system_raw(f"""
echo $( for i in `seq 10`; do
  capture_tpu_profile --service_addr $COLAB_TPU_ADDR --logdir={LOG_DIR}
  sleep 10
done ) &
""")

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/client
+ localtunnel@1.9.1
added 54 packages from 34 contributors in 3.679s
    100% |████████████████████████████████| 4.1MB 3.2MB/s 


In [3]:
! cat url.txt
! rm url.txt

your url is: https://witty-chipmunk-67.localtunnel.me


In [0]:
import numpy as np
import tensorflow as tf
from data_util_tpu import RandomAugmentSequence, center_to_corner_box
from data_util_tpu import get_anchors, extraction_boxes, show_image_labels

In [0]:
# 全データの読み込み
images = np.load('etl9b/images.npy', mmap_mode='r')
boxes = np.load('etl9b/boxes.npy', mmap_mode='r')
labels = np.load('etl9b/labels.npy', mmap_mode='r')

In [0]:
# パラメータ
n_concat = 3
batch_size = 1024
image_size = [64, 64]
input_shape = np.array([image_size[0]*1.5, image_size[1]*(n_concat)], dtype=np.int64)
anchors_path = 'model_data/yolo_anchors.txt'
anchors = get_anchors(anchors_path)
num_classes = 3036
log_dir = 'model_weight/'
model_name = 'weights5.h5'

num_cores = 8 # tpuのコア数

In [0]:
#  データのSequenceの取得
split_idx = 190 * 3036
max_idx = 200*3036
train_idx = range(split_idx)
test_idx = range(split_idx, max_idx)

r_translate = 0.3
r_scale = 0.3
r_aspect_rate = 0.3

train_data_sequence = RandomAugmentSequence(
    train_idx, images, boxes, labels, batch_size, image_size, input_shape,
    anchors, num_classes, n_concat, r_translate, r_scale, r_aspect_rate)
test_data_sequence = RandomAugmentSequence(
    test_idx, images, boxes, labels, batch_size, image_size, input_shape,
    anchors, num_classes, n_concat, r_translate, r_scale, r_aspect_rate)

In [8]:
# modelの定義
from model_util_tpu import create_model


model = create_model(input_shape, int(batch_size/num_cores), anchors, num_classes, load_pretrained=False, freeze_body=2,
                     weights_path='model_data/yolo_weights.h5', num_channels=1)
initial_epochs = 0

# from keras.models import load_model

# model = load_model(log_dir + 'weights.h5')
# initial_epochs = 1

Instructions for updating:
Colocations handled automatically by placer.
Create YOLOv3 model with 9 anchors and 3036 classes.


In [9]:
# modelをtpuモデルに変換

import os

model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.20.76.194:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 11721467303704732290)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6040533495564723491)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 2252777935029535128)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1

In [0]:
# modelのコンパイル
from yolo3.model import yolo_loss
from tensorflow.python.keras.optimizers import TFOptimizer
from tensorflow.train import AdamOptimizer

model.compile(optimizer=TFOptimizer(AdamOptimizer(learning_rate=1e-3)), loss={
            'lambda_2': lambda y_true, y_pred: y_pred[0]})

In [11]:
# モデルの学習
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, EarlyStopping, CSVLogger

# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
csv_logger = CSVLogger('history.csv')
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

model.fit_generator(train_data_sequence,
                    validation_data=test_data_sequence,
                    epochs=100,
                    initial_epoch=initial_epochs)
                   # callbacks=[csv_logger])

Epoch 1/100
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(128,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(128, 96, 192, 1), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(128, 3, 6, 3, 6), dtype=tf.float32, name='input_2_10'), TensorSpec(shape=(128, 6, 12, 3, 6), dtype=tf.float32, name='input_3_10'), TensorSpec(shape=(128, 12, 24, 3, 6), dtype=tf.float32, name='input_4_10'), TensorSpec(shape=(128, 1), dtype=tf.float32, name='lambda_2_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:Remapping placeholder for input_2
INFO:tensorflow:Remapping placeholder for input_3
INFO:tensorflow:Remapping placeholder for input_4
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 129.5239667892456 secs
INFO:tensorflow:Setting weights on TPU model.
  1/187 [.....................

UnavailableError: ignored

In [0]:
model.save(log_dir + model_name)

In [0]:
# memory
import sys
threshould = 100

!free -h
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > threshould:
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))